Set Up API KEYS

In [48]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv(override=True)

# Access the secrets
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ELEVENLABS_API_KEY = os.getenv("ELEVENLABS_API_KEY")

print(f"OpenAI Key: {OPENAI_API_KEY[:5]}... (Hidden for security)")
print(f"11Labs Key: {ELEVENLABS_API_KEY[:5]}... (Hidden for security)")


OpenAI Key: sk-pr... (Hidden for security)
11Labs Key: sk_06... (Hidden for security)


Initialize OpenAI

In [49]:
from openai import OpenAI
client = OpenAI()

Record Audio Clip (10s test)

In [40]:
import sounddevice as sd
import numpy as np
import wave

# List available audio devices
devices = sd.query_devices()
input_devices = [d for i, d in enumerate(devices) if d["max_input_channels"] > 0]

# Print available input devices
print("\nAvailable Input Devices:")
for i, device in enumerate(input_devices):
    print(f"{i}: {device['name']} (Max Channels: {device['max_input_channels']})")

# Prompt user for device selection
device_index = int(input("\nEnter the number of the device you want to use: "))
selected_device = input_devices[device_index]

# Set the selected device
sd.default.device = selected_device["index"]
channels = min(selected_device["max_input_channels"], 2)  # Use max available channels, up to 2

print(f"\nUsing device: {selected_device['name']} with {channels} channels.\n")

# Recording parameters
duration = 10  # seconds
sample_rate = 44100

print("Recording...")
audio_data = sd.rec(int(sample_rate * duration), samplerate=sample_rate, channels=channels, dtype=np.int16)
sd.wait()
print("Recording complete.")

# Save to a WAV file
filename = "user_recording.wav"
with wave.open(filename, 'wb') as wf:
    wf.setnchannels(channels)
    wf.setsampwidth(2)
    wf.setframerate(sample_rate)
    wf.writeframes(audio_data.tobytes())

print(f"Saved recording as {filename}")



Available Input Devices:
0: Microsoft Sound Mapper - Input (Max Channels: 2)
1: Microphone (3- Afterglow AG9+ W (Max Channels: 1)
2: Microphone (Realtek(R) Audio) (Max Channels: 2)
3: Primary Sound Capture Driver (Max Channels: 2)
4: Microphone (3- Afterglow AG9+ Wireless Headset) (Max Channels: 1)
5: Microphone (Realtek(R) Audio) (Max Channels: 2)
6: Microphone (Realtek(R) Audio) (Max Channels: 2)
7: Microphone (3- Afterglow AG9+ Wireless Headset) (Max Channels: 1)
8: Microphone (Afterglow AG9+ Wireless Headset) (Max Channels: 1)
9: Stereo Mix (Realtek HD Audio Stereo input) (Max Channels: 2)
10: Microphone (Realtek HD Audio Mic input) (Max Channels: 2)
11: Headset (@System32\drivers\bthhfenum.sys,#2;%1 Hands-Free AG Audio%0
;(MusiBaby-M68)) (Max Channels: 1)
12: Microphone (Voicemod VAD Wave) (Max Channels: 2)
13: Headset (@System32\drivers\bthhfenum.sys,#2;%1 Hands-Free AG Audio%0
;(YT18)) (Max Channels: 1)
14: Headset (@System32\drivers\bthhfenum.sys,#2;%1 Hands-Free AG Audio%0
;(

Transcribe Audio to Text - WhisperAI

In [41]:
def transcribe_audio(audio_path):
    with open(audio_path, "rb") as audio_file:
        transcript = client.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file
        )
    return transcript.text

transcribed_text = transcribe_audio("user_recording.wav")
print("Transcribed Text:", transcribed_text)

Transcribed Text: La-da-dee-dee-dee. Hi, my name is Greg and this is Greg Radio, the best place that you can find any kind of radio ever. It's amazing.


Generate AI Response with transcribed_text

In [42]:
def get_gpt_response(prompt):
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

response_text = get_gpt_response(transcribed_text)
print("GPT Response:", response_text)

GPT Response: Welcome back to another episode! Today, we have a variety of content lined up. As you all know, we love to delve into various topics here at Greg Radio; we take you from the caves of history to the peaks of future technology, jazz up your mood with some music, fill your heart with inspiring stories, and make you smarter with intriguing trivia.

We have a brilliant interview lined up for 10 AM, where we are talking to renowned author, Rebecca York, who recently won the Booker Prize for her thought-provoking book 'Eclipse Whisper'. She will share insights into her writing process, inspirations, and give us a sneak peek into her next novel.

Masterchef Ruth Madison will join us at 1 PM and take us through some delectable recipes right from her cookbook. Stay tuned while she adds a sizzle to your afternoon with her signature dish.

Do you feel the need to let out some steam after a long day at work? Then, don't forget to join our popular evening segment - 'Rant it out' at 6 P

11 Labs text-to-speech

In [54]:
import requests

ELEVENLABS_API_KEY = os.getenv("ELEVENLABS_API_KEY")

print("Loaded API Key:", ELEVENLABS_API_KEY[:5], "********")

# API request to get voices
def get_voice_ids():
    url = "https://api.elevenlabs.io/v1/voices"
    headers = {"xi-api-key": ELEVENLABS_API_KEY}

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        voices = response.json()["voices"]
        for voice in voices:
            print(f"Voice Name: {voice['name']}, Voice ID: {voice['voice_id']}")
        return voices
    else:
        print("Error fetching voices:", response.text)
        return None

# Get valid voice IDs
voice_options = get_voice_ids()
voices = [v for i, v in enumerate(voice_options) if v["voice_id"] in v]

print("\nVoice Options")
for i, voice in enumerate(voice_options):
    print(f"{i}: {voice['name']} (ID: {voice['voice_id']})")

# Prompt user for device selection
voice_index = int(input("\nEnter the number of the voice you want to use: "))
selected_voice = voice_options[voice_index]["voice_id"]


def text_to_speech_11labs(text, voice_id=selected_voice):
    url = f"https://api.elevenlabs.io/v1/text-to-speech/{voice_id}"
    headers = {"xi-api-key": ELEVENLABS_API_KEY, "Content-Type": "application/json"}
    data = {"text": text, "model_id": "eleven_monolingual_v1"}

    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        with open("output_speech.mp3", "wb") as f:
            f.write(response.content)
        print("Generated Speech: output_speech.mp3")
    else:
        print("Error:", response.text)

text_to_speech_11labs(response_text)


Loaded API Key: sk_06 ********
Voice Name: Aria, Voice ID: 9BWtsMINqrJLrRacOk9x
Voice Name: Roger, Voice ID: CwhRBWXzGAHq8TQ4Fs17
Voice Name: Sarah, Voice ID: EXAVITQu4vr4xnSDxMaL
Voice Name: Laura, Voice ID: FGY2WhTYpPnrIDTdsKH5
Voice Name: Charlie, Voice ID: IKne3meq5aSn9XLyUdCD
Voice Name: George, Voice ID: JBFqnCBsd6RMkjVDRZzb
Voice Name: Callum, Voice ID: N2lVS1w4EtoT3dr4eOWO
Voice Name: River, Voice ID: SAz9YHcvj6GT2YYXdXww
Voice Name: Liam, Voice ID: TX3LPaxmHKxFdv7VOQHJ
Voice Name: Charlotte, Voice ID: XB0fDUnXU5powFXDhCwa
Voice Name: Alice, Voice ID: Xb7hH8MSUJpSbSDYk0k2
Voice Name: Matilda, Voice ID: XrExE9yKIg1WjnnlVkGX
Voice Name: Will, Voice ID: bIHbv24MWmeRgasZH58o
Voice Name: Jessica, Voice ID: cgSgspJ2msm6clMCkdW9
Voice Name: Eric, Voice ID: cjVigY5qzO86Huf0OWal
Voice Name: Chris, Voice ID: iP95p4xoKVk53GoZ742B
Voice Name: Brian, Voice ID: nPczCjzI2devNBz1zQrb
Voice Name: Daniel, Voice ID: onwK4e9ZLuTAKqWW03F9
Voice Name: Lily, Voice ID: pFZP5JQG7iQjIQuC4Bku
Voice Name:

Download Audio - in Notebook

In [55]:
import IPython.display as display

# Display a download link in Jupyter Notebook
def download_audio(filename="output_speech.mp3"):
    return display.FileLink(filename)

download_audio()


C:\Users\gregc\PycharmProjects\VoiceGPTtesting\notebooks\output_speech.mp3